In [1]:
import numpy as np

def createDataSet():
    group = np.array([[1.0, 1.1], [1.0, 1.0], [0, 0], [0, 0.1]])
    labels = ['A', 'A', 'B', 'B']
    return group, labels

In [2]:
group, labels = createDataSet()

print('group:', group)
print('labels:', labels)  # 输出数值

group: [[1.  1.1]
 [1.  1. ]
 [0.  0. ]
 [0.  0.1]]
labels: ['A', 'A', 'B', 'B']


In [3]:
# 在 Jupyter Notebook 单元格中执行，下载并解压数据。
!wget -nc "http://labfile.oss.aliyuncs.com/courses/777/digits.zip"
# 解压缩
!unzip -o digits.zip

--2021-06-23 19:36:14--  http://labfile.oss.aliyuncs.com/courses/777/digits.zip
Resolving labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)... 47.110.177.159
Connecting to labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)|47.110.177.159|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 840301 (821K) [application/zip]
Saving to: 'digits.zip'

     0K .......... .......... .......... .......... ..........  6% 1.02M 1s
    50K .......... .......... .......... .......... .......... 12% 2.05M 1s
   100K .......... .......... .......... .......... .......... 18% 6.28M 0s
   150K .......... .......... .......... .......... .......... 24% 23.2M 0s
   200K .......... .......... .......... .......... .......... 30% 2.21M 0s
   250K .......... .......... .......... .......... .......... 36% 1.08M 0s
   300K .......... .......... .......... .......... .......... 42% 34.6M 0s
   350K .......... .......... .......... .......... .......... 48% 18.8M 0s
   400K .......... ..

Archive:  digits.zip
   creating: digits/               
   creating: digits/trainingDigits/
  inflating: digits/trainingDigits/7_175.txt  
  inflating: digits/trainingDigits/3_51.txt  
  inflating: digits/trainingDigits/7_143.txt  
  inflating: digits/trainingDigits/0_17.txt  
  inflating: digits/trainingDigits/2_111.txt  
  inflating: digits/trainingDigits/0_102.txt  
  inflating: digits/trainingDigits/8_36.txt  
  inflating: digits/trainingDigits/3_104.txt  
  inflating: digits/trainingDigits/6_105.txt  
  inflating: digits/trainingDigits/3_30.txt  
  inflating: digits/trainingDigits/5_72.txt  
  inflating: digits/trainingDigits/3_41.txt  
  inflating: digits/trainingDigits/0_169.txt  
  inflating: digits/trainingDigits/9_173.txt  
  inflating: digits/trainingDigits/3_80.txt  
  inflating: digits/trainingDigits/9_37.txt  
  inflating: digits/trainingDigits/3_187.txt  
  inflating: digits/trainingDigits/6_91.txt  
  inflating: digits/trainingDigits/9_92.txt  
  inflating: digits/trai

In [7]:
!type digits\testDigits\0_1.txt

00000000000000011000000000000000
00000000000111111110000000000000
00000000001111111111100000000000
00000000001111111111110000000000
00000000011111111111111000000000
00000000011111100011111000000000
00000000111110000001111000000000
00000000111110000001111100000000
00000000111110000000111110000000
00000001111110000000111110000000
00000001111110000000011111000000
00000001111110000000001111000000
00000001111110000000001111100000
00000001111100000000001111000000
00000001111000000000001111000000
00000001111000000000001111000000
00000001111000000000000111000000
00000000111100000000000111000000
00000000111100000000000111000000
00000000111100000000000111000000
00000001111000000000011110000000
00000001111000000000011110000000
00000000111000000000011110000000
00000000111110000011111110000000
00000000111110001111111100000000
00000000111111111111111000000000
00000000011111111111111000000000
00000000111111111111100000000000
00000000011111111111000000000000
00000000001111111000000000000000
0000000000

In [8]:
def img2vector(filename):
    # 创建向量
    returnVect = np.zeros((1, 1024))
    # 打开数据文件，读取每行内容
    fr = open(filename)
    for i in range(32):
        # 读取每一行
        lineStr = fr.readline()
        # 将每行前 32 字符转成 int 存入向量
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j])
            
    return returnVect

In [9]:
img2vector('digits/testDigits/0_1.txt')

array([[0., 0., 0., ..., 0., 0., 0.]])

In [10]:
import operator


def classify0(inX, dataSet, labels, k):
    
    """
    参数: 
    - inX: 用于分类的输入向量
    - dataSet: 输入的训练样本集
    - labels: 样本数据的类标签向量
    - k: 用于选择最近邻居的数目
    """
    
    # 获取样本数据数量
    dataSetSize = dataSet.shape[0]

    # 矩阵运算，计算测试数据与每个样本数据对应数据项的差值
    diffMat = np.tile(inX, (dataSetSize, 1)) - dataSet

    # sqDistances 上一步骤结果平方和
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1)

    # 取平方根，得到距离向量
    distances = sqDistances**0.5

    # 按照距离从低到高排序
    sortedDistIndicies = distances.argsort()
    classCount = {}

    # 依次取出最近的样本数据
    for i in range(k):
        # 记录该样本数据所属的类别
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1

    # 对类别出现的频次进行排序，从高到低
    sortedClassCount = sorted(
        classCount.items(), key=operator.itemgetter(1), reverse=True)

    # 返回出现频次最高的类别
    return sortedClassCount[0][0]

In [11]:
group, labels = createDataSet()
classify0([0, 0], group, labels, 3)

'B'

In [12]:
from os import listdir


def handwritingClassTest():
    # 样本数据的类标签列表
    hwLabels = []

    # 样本数据文件列表
    trainingFileList = listdir('digits/trainingDigits')
    m = len(trainingFileList)

    # 初始化样本数据矩阵（M*1024）
    trainingMat = np.zeros((m, 1024))

    # 依次读取所有样本数据到数据矩阵
    for i in range(m):
        # 提取文件名中的数字
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)

        # 将样本数据存入矩阵
        trainingMat[i, :] = img2vector(
            'digits/trainingDigits/%s' % fileNameStr)

    # 循环读取测试数据
    testFileList = listdir('digits/testDigits')

    # 初始化错误率
    errorCount = 0.0
    mTest = len(testFileList)

    # 循环测试每个测试数据文件
    for i in range(mTest):
        # 提取文件名中的数字
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])

        # 提取数据向量
        vectorUnderTest = img2vector('digits/testDigits/%s' % fileNameStr)

        # 对数据文件进行分类
        classifierResult = classify0(vectorUnderTest, trainingMat, hwLabels, 3)

        # 打印 K 近邻算法分类结果和真实的分类
        print("测试样本 %d, 分类器预测: %d, 真实类别: %d" %
              (i+1, classifierResult, classNumStr))

        # 判断K 近邻算法结果是否准确
        if (classifierResult != classNumStr):
            errorCount += 1.0

    # 打印错误率
    print("\n错误分类计数: %d" % errorCount)
    print("\n错误分类比例: %f" % (errorCount/float(mTest)))

In [13]:
handwritingClassTest()

测试样本 1, 分类器预测: 0, 真实类别: 0
测试样本 2, 分类器预测: 0, 真实类别: 0
测试样本 3, 分类器预测: 0, 真实类别: 0
测试样本 4, 分类器预测: 0, 真实类别: 0
测试样本 5, 分类器预测: 0, 真实类别: 0
测试样本 6, 分类器预测: 0, 真实类别: 0
测试样本 7, 分类器预测: 0, 真实类别: 0
测试样本 8, 分类器预测: 0, 真实类别: 0
测试样本 9, 分类器预测: 0, 真实类别: 0
测试样本 10, 分类器预测: 0, 真实类别: 0
测试样本 11, 分类器预测: 0, 真实类别: 0
测试样本 12, 分类器预测: 0, 真实类别: 0
测试样本 13, 分类器预测: 0, 真实类别: 0
测试样本 14, 分类器预测: 0, 真实类别: 0
测试样本 15, 分类器预测: 0, 真实类别: 0
测试样本 16, 分类器预测: 0, 真实类别: 0
测试样本 17, 分类器预测: 0, 真实类别: 0
测试样本 18, 分类器预测: 0, 真实类别: 0
测试样本 19, 分类器预测: 0, 真实类别: 0
测试样本 20, 分类器预测: 0, 真实类别: 0
测试样本 21, 分类器预测: 0, 真实类别: 0
测试样本 22, 分类器预测: 0, 真实类别: 0
测试样本 23, 分类器预测: 0, 真实类别: 0
测试样本 24, 分类器预测: 0, 真实类别: 0
测试样本 25, 分类器预测: 0, 真实类别: 0
测试样本 26, 分类器预测: 0, 真实类别: 0
测试样本 27, 分类器预测: 0, 真实类别: 0
测试样本 28, 分类器预测: 0, 真实类别: 0
测试样本 29, 分类器预测: 0, 真实类别: 0
测试样本 30, 分类器预测: 0, 真实类别: 0
测试样本 31, 分类器预测: 0, 真实类别: 0
测试样本 32, 分类器预测: 0, 真实类别: 0
测试样本 33, 分类器预测: 0, 真实类别: 0
测试样本 34, 分类器预测: 0, 真实类别: 0
测试样本 35, 分类器预测: 0, 真实类别: 0
测试样本 36, 分类器预测: 0, 真实类别: 0
测试样本 37, 分类器预测: 0, 真实类别: 0
测试样本 38, 分